In [23]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

In [24]:
import ipynb.fs.full.TrainTest as TrainTest

p = 30 # want this to be high
w = 4  # want this to be low
cw = 9 # want this to be high
data = np.load('./data/' + 'p' + str(p) + '_w' + str(w) + '_cw' + str(cw) + '.npy')

x_train, y_train, x_test, y_test = TrainTest.get_train_test(data, 0.8)

In [25]:
dt = DecisionTreeClassifier()

param_options = {
    'criterion' : ['gini', 'entropy'],
    'splitter' : ['best', 'random'],
    'max_depth' : [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4],
#     'min_weight_fraction_leaf' : [],
    'max_features' : ['auto', 'log2', 3, 4, 5, None],
#     'random_state' : [],
#     'max_leaf_nodes' : [],
#     'min_impurity_decrease' : [],
#     'min_impurity_split' : [],
    'class_weight' : [None, 'balanced'],
    'presort' : [True, False]
}

dt_random = RandomizedSearchCV(estimator = dt, param_distributions = param_options, 
    n_iter = 100, cv = 10, verbose=1, random_state=42, n_jobs = -1)

dt_random.fit(x_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.9s finished


RandomizedSearchCV(cv=10, error_score='raise',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'splitter': ['best', 'random'], 'presort': [True, False], 'criterion': ['gini', 'entropy'], 'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'min_samples_leaf': [1, 2, 4], 'max_features': ['auto', 'log2', 3, 4, 5, None], 'class_weight': [None, 'balanced'], 'min_samples_split': [2, 5, 10]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [26]:
best = dt_random.best_params_
best

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 100,
 'max_features': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'presort': False,
 'splitter': 'best'}

In [27]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

dt = DecisionTreeClassifier(**best)

avg_acc = 0
avg_prec = 0
avg_f1 = 0
num = 20
for i in range(num):
    dt.fit(x_train, y_train)
    
    y_pred = dt.predict(x_test)
    
    avg_acc += accuracy_score(y_test, y_pred)
    avg_prec += precision_score(y_test, y_pred)
    avg_f1 += f1_score(y_test, y_pred)

avg_acc /= num
avg_prec /= num
avg_f1 /= num

print('Average accuracy:  ' + str(avg_acc))
print('Average precision: ' + str(avg_prec))
print('Average F1 Score:  ' + str(avg_f1))

Average accuracy:  0.6910126582278482
Average precision: 0.631180179467885
Average F1 Score:  0.6807788558361927
